In [ ]:
!pip install datasets --quiet
!pip install accelerate -U --quiet
!pip install evaluate --quiet
!pip install jiwer -U  -q
!pip install transformers -U -q
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 42.1 MB/s eta 0:

In [ ]:
!git clone https://github.com/f90/jamendolyrics.git --quiet

In [ ]:
!gdown 13bUduvbSf4AowpM5NN-owNKUYdQsrfm_
!unzip -q /content/musan_resampled.zip
!rm /content/musan_resampled.zip

Downloading...
From (original): https://drive.google.com/uc?id=13bUduvbSf4AowpM5NN-owNKUYdQsrfm_
From (redirected): https://drive.google.com/uc?id=13bUduvbSf4AowpM5NN-owNKUYdQsrfm_&confirm=t&uuid=032ea16f-0471-4c34-9fab-3c034cefa62b
To: /content/musan_resampled.zip
100% 1.61G/1.61G [00:24<00:00, 66.2MB/s]


In [ ]:
# !wget https://www.openslr.org/resources/17/musan.tar.gz
# !tar -zxf musan.tar.gz
# !rm -rf /content/musan/music
# !rm -rf /content/musan/speech
# !rm /content/musan.tar.gz

In [ ]:
import torch
import argparse
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import DatasetDict, Audio, load_from_disk, concatenate_datasets
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
from datasets import load_dataset, DatasetDict
import wandb
import os
import numpy as np
import random
import torchaudio
import soundfile
from torch.utils.data import Dataset
import glob
import pandas as pd
import csv
import librosa


In [ ]:
wandb.login()

wandb: Currently logged in as: banfizsombor1999. Use `wandb login --relogin` to force relogin


True

In [ ]:
MODEL_NAME = "openai/whisper-medium"
# LANGUAGE = "en"
FREEZE_FEATURE_ENCODER = False
FREEZE_ENCODER = False

feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_NAME,  task="transcribe")
processor = WhisperProcessor.from_pretrained(MODEL_NAME,task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
if FREEZE_FEATURE_ENCODER:
    model.freeze_feature_encoder()

if FREEZE_ENCODER:
    model.freeze_encoder()
    model.model.encoder.gradient_checkpointing = False

In [ ]:
metadata = pd.read_csv('/content/jamendolyrics/JamendoLyrics.csv')
metadata.head()

,URL,Filepath,Artist,Title,Genre,LicenseType,Language,LyricOverlap,Polyphonic,NonLexical
0,https://www.jamendo.com/track/1559261/give-me-...,HILA_-_Give_Me_the_Same.mp3,HILA,Give Me The Same,Pop,BY-ND,English,False,False,False
1,https://www.jamendo.com/track/1552064/keep-on,Quentin_Hannappe_-_Keep_On.mp3,Quentin Hannappe,Keep On,Pop,BY-NC-ND,English,False,False,False
2,https://www.jamendo.com/track/1537288/back-in-...,Songwriterz_-_Back_In_Time.mp3,Songwriterz,Back In Time,Pop,BY-ND,English,False,False,True
3,https://www.jamendo.com/track/1442030/peyote,Kinematic_-_Peyote.mp3,KINEMATIC,Peyote,Rock,BY-ND,English,False,True,False
4,https://www.jamendo.com/track/1465148/embers,Avercage_-_Embers.mp3,Avercage,Embers,Rock,BY-NC-SA,English,False,False,False


In [ ]:
metadata = metadata.set_index("Filepath")

In [ ]:


SR = 16000
mp3_path = '/content/jamendolyrics/mp3/'
csv_path = '/content/jamendolyrics/annotations/lines/'
file_names = sorted(glob.glob(csv_path + '*.csv'))

gather_n = 4 ################################################### Combine n data samples into one data
len_files = []
start_time = []
end_time = []
audio_types = []
lyrics = []
languages = []
audio_dict = {} # load audio array in advance to use less RAM. key: 0~78, value: np.array

for f in file_names:
    c = pd.read_csv(f)
    len_files.append(len(c))

def preprocess_dataset():
    for num, filename in enumerate(file_names):
        len_csv = len_files[num]
        with open(f"{filename}", "r") as f:
            csv_data = csv.reader(f)
            audio_name = filename.split('/')[-1][:-4] + '.mp3'
            language = metadata.loc[audio_name]["Language"].lower()
            processor.tokenizer.set_prefix_tokens(language=language)

            for i, line in enumerate(csv_data):
                if i== 0: # except first head
                    continue

                if gather_n == 1: # not combine samples
                    st = float(line[0])
                    et = float(line[1])
                    lyric = line[2]
                    lyric = processor.tokenizer(lyric).input_ids

                    start_time.append(st)
                    end_time.append(et)
                    lyrics.append(lyric)
                    audio_types.append(num)

                else:
                    if i%gather_n == 1: # start gathering frames
                        st = float(line[0])
                        et = float(line[1])
                        lyric = line[2]

                    else:
                        et = float(line[1])
                        lyric += (' ' + line[2])

                    if i%gather_n == 0 or i == len_csv: # end of the group or the last line
                        start_time.append(st)
                        end_time.append(et)
                        lyric = processor.tokenizer(lyric).input_ids
                        lyrics.append(lyric)
                        audio_types.append(num)

        f.close()

In [ ]:
preprocess_dataset()

In [ ]:
# Load original audio arrays in advance
for i, filename in enumerate(file_names):
    audio_name = filename.split('/')[-1][:-4] + '.mp3'
    audio_dict[i], _ = librosa.load(mp3_path + audio_name, sr=SR)

trimmed_audios = []
def trim_audio(audio_name, st, et):
    start_frame = SR*st
    end_frame = SR*et
    audio  = audio_dict[audio_name] # sr=44100
    trimmed_audio = audio[int(start_frame):int(end_frame)+1]

    return trimmed_audio

In [ ]:
for audio_type, st, et in zip(audio_types, start_time, end_time):
    trimmed_audios.append(trim_audio(audio_type, st, et))

In [ ]:
# noise_ds = load_dataset('./musan')
noise_ds = load_from_disk('/content/content/musan_resampled')

In [ ]:
trimmed_audios[0]

array([-0.03685588, -0.12578191,  0.02261727, ...,  0.18195425,
        0.09405375,  0.03368825], dtype=float32)

In [ ]:
# Pytorch Dataset
# Have to add noise before feature extraction


def add_noise_to_audio(audio, noise, clean_db, noise_snr):
    noise_db = 10 * np.log10(np.mean(noise ** 2)+1e-4)
    noises = [np.sqrt(10 ** ((clean_db - noise_db - noise_snr) / 10)) * noise]

    if len(noises[0]) < len(audio):
        added = np.pad(noises[0], (0, len(audio)-len(noises[0])), 'constant') + audio
    else:
        added = audio + noises[0][:len(audio)]

    return added

class PairedDataset(Dataset):
    def __init__(self, trimmed_audios, lyrics,noise_ds,snr = 10,random_augment = True):
        self.lyrics = lyrics
        self.trimmed_audios = trimmed_audios
        self.noise_dataset = noise_ds
        self.feature_extractor = feature_extractor
        self.snr = snr

        self.random_augment = random_augment

    def __len__(self):
        return len(self.lyrics)

    def __getitem__(self, index):
        if self.random_augment:
          noise_index = random.randint(0, len(self.noise_dataset) - 1)
        else:
          noise_index = index % len(self.noise_dataset)

        noise_sample = self.noise_dataset[noise_index]
        noise_array = noise_sample['audio']['array'] * 30
        audio_array = self.trimmed_audios[index]

        clean_db = 10 * np.log10(np.mean(audio_array ** 2)+1e-4) ##
        audio_array = audio_array.squeeze()#.numpy()
        noisy_audio = add_noise_to_audio(audio_array, noise_array, clean_db, noise_snr=self.snr)

        audio_processed = self.feature_extractor(noisy_audio, sampling_rate=SR).input_features[0]

        return {'input_features': audio_processed, 'labels': self.lyrics[index]}




In [ ]:
SNR = -3
paired_dataset = PairedDataset(trimmed_audios, lyrics, noise_ds,snr = SNR)

### Approach 1. Use pytorch dataset format (I'm not sure if this format is also available in huggingface Trainer)

In [ ]:
### Approach 1. Use pytorch dataset format (I'm not sure if this format is also available in huggingface Trainer)

import torch
from torch.utils.data.dataset import random_split
len_full_dataset = len(paired_dataset)

# train samples
train_p = 0.8
len_train = int(len_full_dataset * train_p)

# valid samples
valid_p = 0.1
len_valid = int(len_full_dataset * 0.1)

train_dataset, valid_dataset = random_split(paired_dataset, [len_train, len_full_dataset-len_train])
valid_dataset, test_dataset = random_split(valid_dataset, [len_valid, len(valid_dataset)-len_valid])

train_dataset.random_augment = True
valid_dataset.random_augment = False
test_dataset.random_augment = False

In [ ]:
# # Check if the audio aligns with text well
# import IPython.display as ipd
# idx = 20
# print(train_dataset[idx]['labels'])
# ipd.Audio(train_dataset[idx]['input_features'], rate=44100)

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt", padding=True)
        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
metric = evaluate.load("wer")
DO_NORMALIZE_EVAL = True
normalizer = BasicTextNormalizer()
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if DO_NORMALIZE_EVAL:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [ ]:
from transformers.integrations import WandbCallback


def decode_predictions(tokenizer, predictions):
    pred_ids = predictions.predictions
    label_ids = predictions.label_ids
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    return {"labels": label_str, "predictions": pred_str}


class WandbPredictionProgressCallback(WandbCallback):
    """Custom WandbCallback to log model predictions during training.

    This callback logs model predictions and labels to a wandb.Table at each logging step during training.
    It allows to visualize the model predictions as the training progresses.

    Attributes:
        trainer (Trainer): The Hugging Face Trainer instance.
        tokenizer (AutoTokenizer): The tokenizer associated with the model.
        sample_dataset (Dataset): A subset of the validation dataset for generating predictions.
        num_samples (int, optional): Number of samples to select from the validation dataset for generating predictions. Defaults to 100.
    """

    def __init__(self, trainer, tokenizer, val_dataset, num_samples=100, freq=2):
        """Initializes the WandbPredictionProgressCallback instance.

        Args:
            trainer (Trainer): The Hugging Face Trainer instance.
            tokenizer (AutoTokenizer): The tokenizer associated with the model.
            val_dataset (Dataset): The validation dataset.
            num_samples (int, optional): Number of samples to select from the validation dataset for generating predictions. Defaults to 100.
            freq (int, optional): Control the frequency of logging. Defaults to 2.
        """
        super().__init__()
        self.trainer = trainer
        self.tokenizer = tokenizer
        noise_indexes = random.sample(range(0, len(val_dataset)-1), num_samples)
        self.sample_dataset = [val_dataset[noise_index] for noise_index in noise_indexes]
        self.freq = freq


    def on_evaluate(self, args, state, control,  **kwargs):
        super().on_evaluate(args, state, control, **kwargs)
        # control the frequency of logging by logging the predictions every `freq` epochs
        if state.epoch % self.freq == 0:
            # generate predictions
            predictions = self.trainer.predict(self.sample_dataset)
            # decode predictions and labels
            predictions = decode_predictions(self.tokenizer, predictions)
            # add predictions to a wandb.Table
            predictions_df = pd.DataFrame(predictions)
            predictions_df["epoch"] = state.epoch

            # get audio samples from sample_dataset

            # add audio samples as a new column to the DataFrame

            records_table = self._wandb.Table(dataframe=predictions_df)
            # log the table to wandb
            self._wandb.log({"sample_predictions": records_table})


In [ ]:
os.environ["WANDB_PROJECT"] = "Noisy Lyrics Transcription"  # name your W&B project
os.environ["ENTITY"] = "gct634"  # name your W&B project

os.environ["WANDB_LOG_MODEL"] = "end"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"



In [ ]:
OUTPUT_DIR = "whisper-medium-finetuned-jamendo-snr-3"
TRAIN_BATCHSIZE = 32
LEARNING_RATE = 1e-5
WARMUP = 5
EVAL_BATCHSIZE = 5
RESUME_FROM_CKPT = None
NUM_EPOCHS = 50
GRADIENT_CHECKPOINTING= True

In [ ]:
training_args = Seq2SeqTrainingArguments(
        output_dir= OUTPUT_DIR,
        per_device_train_batch_size=TRAIN_BATCHSIZE,
        gradient_accumulation_steps=1,
        learning_rate=LEARNING_RATE,
        warmup_steps=WARMUP,
        gradient_checkpointing=GRADIENT_CHECKPOINTING,
        fp16=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end = True,
        num_train_epochs=NUM_EPOCHS,
        per_device_eval_batch_size=EVAL_BATCHSIZE,
        predict_with_generate=True,
        generation_max_length=225,
        logging_strategy='epoch',
        report_to=["wandb"],
        metric_for_best_model="wer",
        greater_is_better=False,
        optim="adamw_hf",
        resume_from_checkpoint=RESUME_FROM_CKPT,
    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics
)


In [ ]:
early_stopping = EarlyStoppingCallback(early_stopping_patience=10)
progress_callback = WandbPredictionProgressCallback(trainer, processor.tokenizer, valid_dataset, 10,1)



In [ ]:
wandb.init(entity = "gct634", project = "Noisy Lyrics Transcription", name = OUTPUT_DIR
           )

wandb: Currently logged in as: banfizsombor1999 (gct634). Use `wandb login --relogin` to force relogin


In [ ]:
# processor.save_pretrained(training_args.output_dir)

trainer.add_callback(progress_callback)
trainer.add_callback(early_stopping)
print('TRAINING IN PROGRESS...')

trainer.train()
print('DONE TRAINING')
wandb.finish()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


TRAINING IN PROGRESS...


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Epoch,Training Loss,Validation Loss,Wer
1,1.909200,1.329445,62.711864
2,1.034200,0.977458,99.623352
3,0.588300,0.609203,59.039548
4,0.310900,0.612788,49.435028
5,0.218900,0.733746,33.050847
6,0.181100,0.584338,30.131827


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 1